# Cuckoo variant filtering

    date: 2021-10-24
    author: Kevin Murray/Gekkonid Consulting

This notebook takes the variant calls directly out of the Acanthophis
pipeline and filters out obviously poor quality data. The aim of these steps
is not to produce a gold-standard SNP set, rather to produce a smaller,
decent quality SNP set suitable for application-specific filtering at the
 start of each subsequent analysis.

In [ ]:
set -xeuo pipefail

## Statistics generation

First, let's generate some stats for plotting.

In [ ]:
htshax bcfhist \
    -s data/1_filtered/raw_variants_samphist.tsv \
    data/0_raw/mpileup~bwa~cuculus_canorus~all_samples~filtered-default.vcf.gz \
    > data/1_filtered/raw_variants_bcfhist.tsv

## Basic filtering

Here we perform some very basic filtering, removing sites
that are not useful (mostly way too much missing data). 

In [ ]:
bcftools view \
	-m2 -M2 \
	--exclude-uncalled \
	--types snps \
	-i 'QUAL >= 30 &&
	    INFO/DP >= 10 &&
	    INFO/AN >= 3 &&
	    F_MISSING < 0.8' \
	-o data/1_filtered/cuckoo_q30_dp10_an3_mis80.vcf.gz  \
	--threads 48 \
	-Oz \
	data/0_raw/mpileup~bwa~cuculus_canorus~all_samples~filtered-default.vcf.gz
bcftools index data/1_filtered/cuckoo_q30_dp10_an3_mis80.vcf.gz

In [ ]:
htshax bcfhist \
    -s data/1_filtered/cuckoo_q30_dp10_an3_mis80_samphist.tsv \
    data/1_filtered/cuckoo_q30_dp10_an3_mis80.vcf.gz \
    > data/1_filtered/cuckoo_q30_dp10_an3_mis80_bcfhist.tsv

And a more severe set. It seems missingness is the most sensitive axis, and
from the histograms above, we see that the threshold is about 60% where there
is still a reasonable number of SNPs present. We also up the quality to 50
which cuts out a few more.

In [ ]:
bcftools view \
	-m2 -M2 \
	--exclude-uncalled \
	--types snps \
	-i 'QUAL >= 50 &&
	    INFO/DP >= 10 &&
	    INFO/AN >= 6 &&
	    F_MISSING < 0.6' \
	--threads 48 \
	-o data/1_filtered/cuckoo_q50_dp10_an6_mis60.vcf.gz  \
	-Oz \
	data/0_raw/mpileup~bwa~cuculus_canorus~all_samples~filtered-default.vcf.gz
bcftools index data/1_filtered/cuckoo_q50_dp10_an6_mis60.vcf.gz

And finally, a set with a MAF filter to exclude the very large number of
fixed differences to the reference.

In [ ]:
bcftools view \
	-m2 -M2 \
	--exclude-uncalled \
	--types snps \
	-i 'QUAL >= 50 &&
	    INFO/DP >= 10 &&
	    INFO/DP <= 1000 &&
	    INFO/MAF > 0.03 &&
	    F_MISSING < 0.8' \
	--threads 48 \
	-o data/1_filtered/cuckoo_q50_dp10_maf3_mis80.vcf.gz  \
	-Oz \
	data/0_raw/mpileup~bwa~cuculus_canorus~all_samples~filtered-default.vcf.gz
bcftools index data/1_filtered/cuckoo_q50_dp10_maf3_mis80.vcf.gz

In [ ]:
htshax bcfhist \
	-s data/1_filtered/cuckoo_q50_dp10_maf3_mis80_samphist.tsv \
	data/1_filtered/cuckoo_q50_dp10_maf3_mis80.vcf.gz  \
	> data/1_filtered/cuckoo_q50_dp10_maf3_mis80_bcfhist.tsv

# RADseq digest of the reference genome

First, we use radsim-rebed from radhax to digest the genome. This gives us
a BED of RE sites. Specfically, `--length 400` gives us coordinates +-
400bp from each RE site, which is in practice a putative RADseq locus. 

In [ ]:
radsim-digest \
    --genome ../rawdata/reference/GCA_017976375.1_bCucCan1.pri_genomic.fna \
    --ddrad \
    --enzyme HpaII \
    --enzyme2 EcoRI \
    --min 150 \
    --max 500 \
    --output-bed data/1_filtered/radseq-digest-sites.bed

The algorithm of radhax is a bit dumb in that it gives one entry for each
RAD locus, and doesn't merge overlapping sites. So, we subsequently use bedtools merge
to merge overlapping sites. This obviously doesn't equate exactly to a
putative RAD locus, but is close enough for our purposes.

Before merging, we have about 64k sites.

In [ ]:
wc -l data/1_filtered/radseq-digest-sites.bed 

In [ ]:
bedtools merge -i data/1_filtered/radseq-digest-sites.bed -d 50 \
    > data/1_filtered/radseq-digest-sites-dedupe.bed 

After merging, we are left with 56k sites

In [ ]:
wc -l data/1_filtered/radseq-digest-sites-dedupe.bed 

## Putative RAD locus coverage

Now, we calculate coverage in the combined RAD library averaged for each
of these putative RAD loci, so we can filter for loci that are present in
the original RAD libraries. This will give a bit more empirical evidience
to each putative locus.

First we need to merge all RAD libraries into one big BAM

In [ ]:
samtools merge -o data/1_filtered/all_rad_libraries.bam \
    ../data/alignments/samples/bwa/cuculus_canorus/RAD_*.bam
samtools index data/1_filtered/all_rad_libraries.bam 

And now we use mosdepth to calculate median depths per region.

In [ ]:
./scripts/mosdepth \
    --by data/1_filtered/radseq-digest-sites-dedupe.bed \
    --thresholds 1,5,10,50,100,500,1000,5000,10000 \
    --threads 4 \
    --no-per-base \
    --mapq 10 \
    --fast-mode \
    data/1_filtered/all_rad_libraries_mosdepth \
    data/1_filtered/all_rad_libraries.bam 

We then filter regions, keeping only those with at least 50bp
covered at 5x coverage by the original RAD libraries. This might
seem like a low threshold, but it weeds out the ~80%+ of
putative sites with essentially zero coverage in the RAD
library. 

In [ ]:
zcat data/1_filtered/all_rad_libraries_mosdepth.thresholds.bed.gz | \
    awk '$6 > 50{printf("%s\t%s\t%s\n", $1, $2, $3);}' \
    > data/1_filtered/rad_loci_50bp5x.bed

After all these filtering steps, we have a bit over 8k putative
loci, which sounds low, but conservatively so.

In [ ]:
wc -l data/1_filtered/rad_loci_50bp5x.bed

## Variant filtering

And now we finally filter to within the putative RAD loci. We use the
most strict variant filtering set.

In [ ]:
bcftools view \
	-m2 -M2 \
	--exclude-uncalled \
    --regions-file data/1_filtered/rad_loci_50bp5x.bed \
	--types snps \
	-i 'QUAL >= 50 &&
	    INFO/DP >= 10 &&
	    INFO/DP <= 1000 &&
	    INFO/MAF > 0.03 &&
	    F_MISSING < 0.8' \
	--threads 48 \
	-o data/1_filtered/cuckoo_q50_dp10_maf3_mis80_radloci.vcf.gz  \
	-Oz \
	data/0_raw/mpileup~bwa~cuculus_canorus~all_samples~filtered-default.vcf.gz
bcftools index data/1_filtered/cuckoo_q50_dp10_maf3_mis80_radloci.vcf.gz

In [ ]:
htshax bcfhist \
	-s data/1_filtered/cuckoo_q50_dp10_maf3_mis80_radloci_samphist.tsv \
	data/1_filtered/cuckoo_q50_dp10_maf3_mis80_radloci.vcf.gz  \
	> data/1_filtered/cuckoo_q50_dp10_maf3_mis80_radloci_bcfhist.tsv